In [1]:
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trl
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install faiss-gpu-cu12
!pip install -U langchain-community
!pip install -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchai

In [1]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets

from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm

In [2]:
# 모델 수정 필요

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,  # 추가적인 양자화 적용
    bnb_4bit_quant_type="nf4",  # NormalFloat4 양자화 방식 사용
    bnb_4bit_compute_dtype=torch.float16  # FP16으로 계산하여 속도 최적화
)  # 4비트 양자화 활성화

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [3]:
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), 

In [4]:
print("모델 타입:", type(model))
print("토크나이저 타입:", type(tokenizer))

모델 타입: <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'>
토크나이저 타입: <class 'transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast'>


In [6]:
import pandas as pd

# 경로 확인
df = pd.read_csv('/content/drive/MyDrive/데이콘_팀공유/똑바로 다시 시작/test_df.csv', encoding = 'utf-8-sig')

df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
df['인적사고'] = df['인적사고'].str.replace(r'\(.*?\)', '', regex=True)

# best_query
# query = """
# 철근콘크리트공사 작업 중 떨어짐 사고 발생.
# 키워드: 안전난간 미설치, 안전고리 미착용.
# 떨어짐 방지를 위한 안전대책 및 재발 방지 조치는?
# """


# 테스트 데이터 통합 생성
combined_df_data = df.apply(
    lambda row: {
        "question": (
            f" '{row['공종(중분류)']}' 작업 중 '{row['인적사고']}' 발생. \n"
            f"키워드: '{row['사고원인']}' \n"
            f"'{row['인적사고']}' 방지를 위한 조치는?"
        )
    },
    axis=1
)
# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_df_data))
combined_test_data.head()

,question
0,'해체 및 철거공사' 작업 중 '물체에 맞음' 발생. \n키워드: '거푸집 해체작...
1,'토공사' 작업 중 '기타' 발생. \n키워드: '호우 발생에 따른 양수기 침수로...
2,'기타' 작업 중 '넘어짐' 발생. \n키워드: '동바리 자재 설치 도중에 넘어짐...
3,'해체 및 철거공사' 작업 중 '떨어짐' 발생. \n키워드: '해체작업전 위험요인...
4,'가설공사' 작업 중 '끼임' 발생. \n키워드: '스틸폼 양중을 위한 줄걸이 체...


In [7]:
######### 다시 벡터스토어 복원 방법: 연결 끊겼을 때 / 경로 수정 필수
import faiss
import pickle
import numpy as np
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# FAISS 인덱스 로드
faiss_index_path = "/content/drive/MyDrive/faiss_index.bin"
index = faiss.read_index(faiss_index_path)

# 원본 텍스트 로드
faiss_texts_path = "/content/drive/MyDrive/faiss_texts.pkl"
with open(faiss_texts_path, "rb") as f:
    all_splits = pickle.load(f)

# 원본 임베딩 벡터 로드 (새로 추가)
faiss_embeddings_path = "/content/drive/MyDrive/faiss_embeddings.npy"
embeddings_list = np.load(faiss_embeddings_path)

print("FAISS 벡터 스토어 로드 완료!")

# 임베딩 모델 로드 (이전에 사용했던 모델과 동일해야 함)
embedding_model_name = "jhgan/ko-sbert-nli"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# (텍스트, 임베딩) 쌍을 생성하여 복원
text_embedding_pairs = list(zip(all_splits, embeddings_list))

vector_store = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,
    embedding=embedding  # 임베딩 모델 객체 전달
)

# FAISS 인덱스를 다시 적용
vector_store.index = index

print("FAISS 벡터 스토어가 정상적으로 복원되었습니다!")

FAISS 벡터 스토어 로드 완료!


<ipython-input-7-26d88c78fef9>:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS 벡터 스토어가 정상적으로 복원되었습니다!


In [8]:
!pip install kiwipiepy
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 105.8 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818026 sha256=3fdcf933026f2bae7a82ff9d51bd23b74acfb7a88d022a25325f57220d7f91f4
  Stored in directory: /root/.cache/pip/wheels/ca/c8/52/3a539d6e9065b191fe1c215e0203dcc3e00601c0e3d3d39824
Successfully built kiwipiepy_model


In [9]:
from kiwipiepy import Kiwi
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Kiwi 형태소 분석기 초기화 (BM25 토크나이저 용도)
kiwi = Kiwi()

def kiwi_tokenizer(text):
    return [token.form for token in kiwi.tokenize(text)]  # 문장을 형태소 단위로 나눔

# 벡터 검색을 위한 FAISS Vector Store의 데이터 가져오기
docs = [Document(page_content=text) for text in all_splits]  # FAISS의 text 데이터를 Document 형식으로 변환

# FAISS 기반 Retriever 생성
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# BM25 기반 Retriever 생성 (형태소 분석기 이용)
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=kiwi_tokenizer)

# BM25 + FAISS 결합 (앙상블 검색)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],  # 두 개의 검색 시스템 결합
    weights=[0.8, 0.2]  # BM25와 FAISS를 동일한 가중치( 8:2)로 설정
)

In [10]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False, # False 로 하면 같은 입력에 같은 출력
    #temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐 , do_sample = False 일떄 사용 불가
    return_full_text=False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=8  # 배치 크기 지정
)

Device set to use cuda:0


In [11]:
print("모델 타입:", type(model))
print("토크나이저 타입:", type(tokenizer))

모델 타입: <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'>
토크나이저 타입: <class 'transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast'>


In [12]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import pipeline


prompt_template = """
역할: 당신은 건설 안전 전문가입니다. 검색 결과를 바탕으로 질문에 대한 조치 사항을 간결하게 작성하세요.

질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.
검색된 내용에 기반하여 조치를 작성하세요.
목차, 번호, 특수기호 없이 핵심 내용만 서술하세요.
불필요한 부연 설명, 연결어, 주어 제거
반드시 마침표로 끝나는 문장으로 작성

{context}

질문:
{question}

답변:
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # stuff -> 그냥 합쳐서 llm에 전달 / map_reduce -> 합친거 요약해서 llm 전달 / refine -> 순차적으로 문서 추가해서 답변 보완
    retriever=retriever,
    return_source_documents=False, # 생성된 답변과 함께 사용된 소스 문서들도 반환할 수 있습니다. 출처 알려주는 개념
    chain_type_kwargs={"prompt": prompt}
)

In [13]:
print(type(qa_chain))  # 정상적으로 정의된 객체인지 확인

<class 'langchain.chains.retrieval_qa.base.RetrievalQA'>


In [17]:
# 상위 5개 질문 가져오기
test_queries = combined_test_data["question"][:5].tolist()  #

# 답변 생성
responses = []
for idx, query in enumerate(test_queries):

    # 답변 생성
    response = qa_chain.invoke(query)

    # 결과 저장
    responses.append({"질문": query, "답변": response["result"]})

# 결과 출력
df_responses = pd.DataFrame(responses)
df_responses

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warning

,질문,답변
0,'해체 및 철거공사' 작업 중 '물체에 맞음' 발생. \n키워드: '거푸집 해체작...,"거푸집 해체 작업 중 동바리 추락 방지 위해 안전망 설치 및 작업자 간격 유지, 동..."
1,'토공사' 작업 중 '기타' 발생. \n키워드: '호우 발생에 따른 양수기 침수로...,"호우 발생 시 양수기 침수 방지 조치를 확립하고, 양수기 작동상태 점검 실시. 양수..."
2,'기타' 작업 중 '넘어짐' 발생. \n키워드: '동바리 자재 설치 도중에 넘어짐...,"동바리 자재 설치 도중 넘어짐 방지 위해 안전모, 안전화 착용 필수. 작업면 미끄러..."
3,'해체 및 철거공사' 작업 중 '떨어짐' 발생. \n키워드: '해체작업전 위험요인...,"해체 작업 전 비계 상태 점검 및 복구, 작업 구획 설정, 관리 감독자 지정, 안전..."
4,'가설공사' 작업 중 '끼임' 발생. \n키워드: '스틸폼 양중을 위한 줄걸이 체...,스틸폼 양중시 줄걸이 체결부분 철근 또는 보강재 추가 설치 줄걸이 체결부분 강성 확...
